# Model Training

In this notebook, we will ask you a series of questions regarding model selection. Based on your responses, we will ask you to create the ML models that you've chosen. 

The bonus step is completely optional, but if you provide a sufficient third machine learning model in this project, we will add `1000` points to your Kahoot leaderboard score.

**Note**: Use the dataset that you've created in your previous data transformation step (not the original model).

## Questions
Is this a classification or regression task?  

- Classification

Are you predicting for multiple classes or binary classes?  

- Binary

Given these observations, which 2 (or possibly 3) machine learning models will you choose?  

- Logistic Regression, SVM, KNN

## First Model

Using the first model that you've chosen, implement the following steps.

### 1) Create a train-test split

Use your cleaned and transformed dataset to divide your features and labels into training and testing sets. Make sure you’re only using numeric or properly encoded features.  

In [78]:
# imports
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd 
import numpy as np
import time

# using the subset of the entire dataset
transactions = pd.read_csv("../data/bank_transactions_reduced.csv")
transactions.head()

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,CASH_IN,63516.66,C4651324,54465.00,117981.66,C1850473537,0.00,0.00,0,0
1,CASH_IN,159445.40,C1979267367,227600.92,387046.31,C1177644576,751211.13,591765.73,0,0
2,CASH_IN,77739.33,C1886966268,2040.00,79779.33,C1677904805,50056.00,2532.04,0,0
3,CASH_OUT,135334.88,C312314421,48786.00,0.00,C503930792,0.00,135334.88,0,0
4,CASH_IN,118393.70,C728959033,50962.00,169355.70,C279942911,0.00,167607.91,0,0


In [79]:
# load dataset
X = transactions.drop(['isFraud', 'isFlaggedFraud', 'type', 'nameOrig', 'nameDest'], axis=1)
y = transactions['isFraud']
# view first 5 rows of predictors
X.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,63516.66,54465.00,117981.66,0.00,0.00
1,159445.40,227600.92,387046.31,751211.13,591765.73
2,77739.33,2040.00,79779.33,50056.00,2532.04
3,135334.88,48786.00,0.00,0.00,135334.88
4,118393.70,50962.00,169355.70,0.00,167607.91


In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
9254,4991.92,0.00,0.00,0.00,0.00
1561,190509.48,2077257.80,2267767.28,1156520.81,966011.33
1670,63289.56,0.00,0.00,7564302.19,7627591.75
6087,69590.65,0.00,0.00,357084.05,426674.70
6669,154130.51,5133566.98,5287697.49,1273335.54,1119205.04


### 2) Search for best hyperparameters
Use tools like GridSearchCV, RandomizedSearchCV, or model-specific tuning functions to find the best hyperparameters for your first model.

In [81]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=1.0)

lasso.fit(X_train, y_train)
print("Learned coefficients", lasso.coef_, '\n')

Learned coefficients [ 1.73646243e-08  8.75857872e-08 -8.69302550e-08  1.45350714e-08
 -1.47676540e-08] 



c:\Users\user\miniconda3\envs\ds\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.821e+00, tolerance: 1.098e-03
  model = cd_fast.enet_coordinate_descent(


In [82]:
y_train_pred = lasso.predict(X_train)
mse_test = mean_squared_error(y_train, y_train_pred)

print(f"Train MSE (Basic LASSO, alpha=1.0): {mse_test:.2f}")

Train MSE (Basic LASSO, alpha=1.0): 0.00


In [83]:
# RandomSearchCV
alpha_grid = {'alpha': np.linspace(0.01, 10, 100)}

lasso_model = Lasso()

# start the timer
start_time = time.time()

grid_search = RandomizedSearchCV(estimator=lasso_model, param_distributions=alpha_grid, cv=5)
grid_search.fit(X_train, y_train)

# end the timer after we're done fitting
end_time = time.time()
# calculate elapsed time
elapsed_time_grid = end_time - start_time

# Extract the best model from random search
best_alpha_grid = grid_search.best_params_['alpha']
y_test_pred_grid = grid_search.best_estimator_.predict(X_test)
mse_test_grid = mean_squared_error(y_test, y_test_pred_grid)

print(f"RandomizedSearchCV - Best alpha: {best_alpha_grid}")
print(f"RandomizedSearchCV - Test MSE: {mse_test_grid:.2f}")
print(f"RandomizedSearchCV - Time elapsed: {elapsed_time_grid:.2f} seconds")


c:\Users\user\miniconda3\envs\ds\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.123e+00, tolerance: 9.984e-04
  model = cd_fast.enet_coordinate_descent(
c:\Users\user\miniconda3\envs\ds\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.761e+00, tolerance: 8.987e-04
  model = cd_fast.enet_coordinate_descent(
c:\Users\user\miniconda3\envs\ds\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality 

RandomizedSearchCV - Best alpha: 9.394545454545455
RandomizedSearchCV - Test MSE: 0.00
RandomizedSearchCV - Time elapsed: 20.32 seconds


c:\Users\user\miniconda3\envs\ds\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.632e+00, tolerance: 1.098e-03
  model = cd_fast.enet_coordinate_descent(


### 3) Train your model
Select the model with best hyperparameters and generate predictions on your test set. Evaluate your models accuracy, precision, recall, and sensitivity.  

In [ ]:
# Randomly search for the best hyperparameters on a logistic regression model
param_dist = {
    'penalty': ['l1', 'l2'],
    'C': np.linspace(0.01, 1, 100),
    'solver': ['saga'], 
    'max_iter': [10000]
}

model = RandomizedSearchCV(LogisticRegression(), param_distributions=param_dist, cv=5, scoring='accuracy', random_state=42)
model.fit(X_train, y_train)

# Best model from random search
best_params_random = model.best_params_
best_score_random = model.best_score_

print(f"RandomizedSearchCV - Best Params: {best_params_random}")
print(f"RandomizedSearchCV - Cross-Val Accuracy: {best_score_random:.2f}")

RandomizedSearchCV - Best Params: {'solver': 'saga', 'penalty': 'l2', 'max_iter': 10000, 'C': np.float64(0.48000000000000004)}
RandomizedSearchCV - Cross-Val Accuracy: 1.00


In [ ]:
# Use the best model found from RandomizedSearchCV to predict on unseen test data

# extract the best estimator
best_log = model.best_estimator_

# predict on testing data
log_predictions = best_log.predict(X_test)

# evaluate its accuracy
test_score = accuracy_score(log_predictions, y_test)

print(f"RandomizedSearchCV - Coefficients: {best_log.coef_}")
print(f"RandomizedSearchCV - Test Accuracy: {test_score:.2f}")

RandomizedSearchCV - Coefficients: [[-6.71296237e-06 -6.06840396e-06 -8.11399855e-06 -4.57650696e-06
  -6.79674871e-06]]
RandomizedSearchCV - Test Accuracy: 1.00


In [ ]:
# Import for metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

# generate a confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
# calculate all measures of accuracy
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# calculate specificity by hand
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall (Sensitivity): {recall:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"F1 Score: {f1:.2f}")

Confusion Matrix:
 [[1996    0]
 [   4    0]]
Accuracy: 1.00
Precision: 0.00
Recall (Sensitivity): 0.00
Specificity: 1.00
F1 Score: 0.00


c:\Users\user\miniconda3\envs\ds\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Second Model

Create a second machine learning object and rerun steps (2) & (3) on this model. Compare accuracy metrics between these two models. Which handles the class imbalance more effectively?

Create as many code-blocks as needed.

### (Bonus/Optional) Third Model

Create a third machine learning model and rerun steps (2) & (3) on this model. Which model has the best predictive capabilities? 

Create as many code-blocks as needed.